In [ ]:
import utils
import numpy as np
import cv2
import open3d as o3d
import time
import os
import pandas as pd

In [ ]:
# MODEL_NAME = 'PSMNet'
# MODEL_PATH = 'PSMNet\\pretrained\\pretrained_sceneflow_new.tar'
MODEL_NAME = 'RAFT-Stereo'
MODEL_PATH = "RAFT-Stereo/models/raftstereo-realtime.pth"

FRAME_NAME = "frame_1.png"

TEST_LEFT_IMG = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images\cam1_cropped\\frame_1.png"
TEST_RIGHT_IMG = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images\cam2_cropped\\frame_1.png"

# stereo_params is from SystemCalibration (npz file with K/D/R/T/R_L/R_R/P_L/P_R/Q)
stereo_params = np.load('stereo_params.npz')
IMG_SIZE = (894, 714)

In [ ]:
# Load model
model = utils.load_model(MODEL_NAME, MODEL_PATH)

In [ ]:
# Calculate Undistortion and Rectification Remapping
K_L = stereo_params['K_L']
D_L = stereo_params['D_L']
K_R = stereo_params['K_R']
D_R = stereo_params['D_R']
R_L = stereo_params['R_L']
R_R = stereo_params['R_R']
P_L = stereo_params['P_L']
P_R = stereo_params['P_R']
Q = stereo_params['Q']

mapL1, mapL2 = cv2.initUndistortRectifyMap(K_L, D_L, R_L, P_L, IMG_SIZE, cv2.CV_16SC2)
mapR1, mapR2 = cv2.initUndistortRectifyMap(K_R, D_R, R_R, P_R, IMG_SIZE, cv2.CV_16SC2)

In [ ]:
imgL = cv2.imread(TEST_LEFT_IMG,  cv2.IMREAD_COLOR)
imgR = cv2.imread(TEST_RIGHT_IMG, cv2.IMREAD_COLOR)

# Disparity (use utils to keep the pipeline consistent)
disp = utils.compute_disparity(
            MODEL_NAME, model,
            TEST_LEFT_IMG, TEST_RIGHT_IMG,
            mapL1, mapL2, mapR1, mapR2
        )

# Point cloud (rectified-left frame)
pcd = utils.generate_pointcloud(
            MODEL_NAME, disp, TEST_LEFT_IMG, mapL1, mapL2, stereo_params['Q']
        )


In [ ]:
# CT stl path
STL_FILES = []
STL_PATH = "G:\My Drive\Dataset\stl"
for file in os.listdir(STL_PATH):
    if file.endswith('.vtk') or file.endswith('.stl'):
        STL_FILES.append(file)
print(STL_FILES)

BF_to_EN_test = np.load("BF_to_EN_test.npy")
BF_to_EN_old  = np.load("BF_to_EN_old.npy")

CT_to_OT = np.array([[-0.0102021,-0.998141,0.0600825,-67.1015],
                        [0.99575,-0.0156413,-0.0907656,-492.36],
                        [0.0915366,0.0589012,0.994058,1882.09],
                        [0,0,0,1]])

In [ ]:
OT_TRANS_PATH = "G:\My Drive\Dataset\davinci_Xi_05282025\KidneyORXI\calibration_images" + "\\transforms.csv"
OT_transformations = pd.read_csv(OT_TRANS_PATH)
temp = OT_transformations[OT_transformations[' frame name'] == FRAME_NAME]
OT_to_BF_raw = temp[temp['Tool Type'] == 'E']
OT_to_BF_qt = OT_to_BF_raw[['q0','qx', 'qy', 'qz', 'tx', 'ty', 'tz']]

In [ ]:
pcd, mesh_CT = utils.transform_CTMR(STL_PATH, STL_FILES, OT_to_BF_qt, BF_to_EN_test, pcd, CT_to_OT)

In [ ]:
utils.visualize_pcd_mesh(
    pcd, mesh_CT,
    front=(0.0, 0.0, 0.001),
    lookat=(0.0, 0.0, 0.0),
    up=(0.0, 1.0, 0.0),
    zoom=0.7
)